In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['I hear you callin\', "Here I come baby"', 'To save you, oh oh', "Baby no more stallin'"]


In [ ]:
glob() 함수로 lyrics 디렉토리 안에 있는 모든 파일을 읽어온다.

txt 파일을 raw_corpus 리스트에 문장 단위로 이어 붙인다.

In [2]:
import re

# 입력된 문장
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6

    return sentence

https://dojang.io/mod/page/view.php?id=2438
    
re.sub으로 교체해줘야 할 문자열들을 교체해준다. 공백을 지우고, 모든 문자들을 소문자처리하고, 각 문장의 앞 뒤에 start와 end의 패딩을 저장해준다.  


In [3]:
# 정제 함수를 이용해 정제 데이터 구해보기
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: 
        continue
    if len(sentence.split()) > 15: # 토큰의 개수가 15개 넘어가면 학습 데이터에서 제외
        continue
    corpus.append(preprocess_sentence(sentence))

corpus[:10]

['<start> i hear you callin , here i come baby <end>',
 '<start> to save you , oh oh <end>',
 '<start> baby no more stallin <end>',
 '<start> these hands have been longing to touch you baby <end>',
 '<start> and now that you ve come around , to seein it my way <end>',
 '<start> you won t regret it baby , and you surely won t forget it baby <end>',
 '<start> it s unbelieveable how your body s calling for me <end>',
 '<start> i can just hear it callin callin for me my body s callin for you <end>',
 '<start> my body s callin for you <end>',
 '<start> my body s callin for you <end>']

길이가 0 혹은 15 이상의 문장들을 넘기고, 
위에서 정의했던 전처리 함수에 문장들을 넣어주고 해당 값을 corpus 변수에 저장한다. 

In [4]:
import tensorflow as tf

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 12000, # 단어장 크기
        filters = ' ',
        oov_token = "<unk>" # 12000 단어에 포함되지 못한 단어
    )
    tokenizer.fit_on_texts(corpus) # 내부 단어장
    tensor = tokenizer.texts_to_sequences(corpus) # corpus를 Tensor로 변환
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post') # 입력 데이터의 시퀀스 길이를 일정하게 맞춰준다

    print(tensor)
    print(tokenizer)

    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2   5 188 ...   0   0   0]
 [  2  10 579 ...   0   0   0]
 [  2  51  38 ...   0   0   0]
 ...
 [  2   5  92 ...   0   0   0]
 [  2   9 157 ...   0   0   0]
 [  2 163  15 ...   0   0   0]]


https://paul-hyun.github.io/nlp-tutorial-02-02-tokenizer/ 
    
토큰화를 이용한 자연어처리는 단어장에 단어를 저장해두고 해당 단어들을 조합하여 의미를 구성하는 것이다. 
토큰화에 있어선 글자, 띄어쓰기, 단어 등으로 하는 방법 등이 있으나 여기선 단어 단위로 나누었다.
각 토큰은 단어의 빈도 등 각 토큰에 대한 정보가 담겨있다. 

In [5]:

import numpy as np
from sklearn.model_selection import train_test_split

src_input = tensor[:, :-1] # tensor에서 마지막 토큰을 잘라내서 소스 문장 생성
tgt_input = tensor[:, 1:] # tensor에서 <start>를 잘라내서 타겟 문장 생성

print(src_input[0])
print(tgt_input[0])



[  2   5 188   7 798   4  93   5  67  51   3   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  5 188   7 798   4  93   5  67  51   3   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [24]:

x_train, x_test, y_train, y_test = train_test_split(src_input, tgt_input,test_size=0.2, random_state=42)
print("Source Train:", x_train.shape)
print("Target Train:", x_test.shape)

Source Train: (134872, 32)
Target Train: (33718, 32)


In [ ]:
train, test 데이터를 0.8: 0.2의 비율로 나눈다. 

In [9]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size,  activation='softmax')  # added activation softmax
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

https://wikidocs.net/22888 

언어의 데이터는 시간의 흐름을 반영하는 시계열 데이터이기 때문에, LSTM을 이용한다. 

싱글 RNN모델의 경우 입력단자가 많아질수록 앞의 데이터를 반영하는 것이 점점 희미해진다는 문제점이 있다. 
왜냐하면 X1을 기반으로 예측한 Y1값이 뒤로 갈수록 가중치에 계속 곱해지게 되는데, 이때의 가중치는 0~1 사이의 가중치이기 때문에 X1의 영향력이 점점 줄어들 수 밖에 없는 것이다. 

따라서 앞의 데이터 또한 어느 정도의 영향력을 보존할 수 있는 LSTM을 사용한다. LSTM은 단기 상태와 장기상태를 두 가자로 나누어, 정보를 처리하기 때문에 장기상태의 정보의 영향력을 보장할 수 있게 된다. 

In [10]:
epochs = 10
batch_size = 256

optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=batch_size, validation_data=(x_test, y_test), epochs=epochs, validation_split=0.25)

Epoch 1/10
396/396 [==============================] - 831s 2s/step - loss: 2.0577 - accuracy: 0.7327 - val_loss: 1.4563 - val_accuracy: 0.7708
Epoch 2/10
396/396 [==============================] - 820s 2s/step - loss: 1.5694 - accuracy: 0.7667 - val_loss: 1.3894 - val_accuracy: 0.7760
Epoch 3/10
396/396 [==============================] - 826s 2s/step - loss: 1.3412 - accuracy: 0.7782 - val_loss: 1.3267 - val_accuracy: 0.7811
Epoch 4/10
396/396 [==============================] - 827s 2s/step - loss: 1.2668 - accuracy: 0.7822 - val_loss: 1.2861 - val_accuracy: 0.7848
Epoch 5/10
396/396 [==============================] - 827s 2s/step - loss: 1.1998 - accuracy: 0.7866 - val_loss: 1.2525 - val_accuracy: 0.7888
Epoch 6/10
396/396 [==============================] - 823s 2s/step - loss: 1.1299 - accuracy: 0.7934 - val_loss: 1.2262 - val_accuracy: 0.7927
Epoch 7/10
396/396 [==============================] - 820s 2s/step - loss: 1.0652 - accuracy: 0.8006 - val_loss: 1.2056 - val_accuracy: 0.7965

In [13]:
# 모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token:
            break
        if test_tensor.shape[1] >= max_len: 
            break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated



입력받은 문자의 다음 문자로 올 확률이 가장 높은 단어를 택하고 결과값에 이어붙인다. 이후 end 토큰이 나오거나 문장 사이즈를 넘기기전까지 계속 반복한다. 

In [16]:

generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

In [18]:
generate_text(model, tokenizer, init_sentence="<start> i want", max_len=20)

'<start> i want to dig my way to hell <end> '

In [19]:
generate_text(model, tokenizer, init_sentence="<start> i hate", max_len=20)

'<start> i hate the way you lie <end> '

In [20]:
generate_text(model, tokenizer, init_sentence="<start> love", max_len=20)

'<start> love me , love me <end> '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> what", max_len=20)

'<start> what you want be what you want <end> '

In [22]:
generate_text(model, tokenizer, init_sentence="<start> in the morning", max_len=20)

'<start> in the morning , i m a go getter <end> '

In [23]:
generate_text(model, tokenizer, init_sentence="<start> go", max_len=20)

'<start> go , get ur freak on <end> '

#회고 
꽤나 그럴싸하게 문장이 나온다. 다만 아쉬운 것은 과적합의 흔적이 보인다는 것이다. love me 뒤에 또 love me가 온다거나, go get ur freak on 등은 하나의 노래에서 나온 가사들로 보이기 때문이다. 다음엔 한국어 가사를 모아서 해보면 재밌을 것 같다. 하지만 데이터 모으기가 쉽지 않겠지... 크롤링 등 데이터를 모으는 기법들에 대해서 공부를 해보아야겠다. 하면 할수록 데이터의 중요성에 대해 깨닫게 된다. 